In [12]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

CUDA Available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformations

In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),  # Normalize
    transforms.RandomHorizontalFlip(),  # Augmentation
    transforms.RandomCrop(32, padding=4)
])

# Load CIFAR-10 dataset

In [22]:
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True, download=False, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./datasets', train=False, download=False, transform=transform)

# Create DataLoaders

In [25]:
batch_size = 128
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Train the Model

In [42]:
import torch.optim as optim
from models.cnn import CNN

In [48]:
# Load model
model = CNN().to(device)

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [50]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")

Epoch [1/10], Loss: 1.6567
Epoch [2/10], Loss: 1.2831
Epoch [3/10], Loss: 1.1162
Epoch [4/10], Loss: 1.0096
Epoch [5/10], Loss: 0.9388
Epoch [6/10], Loss: 0.8793
Epoch [7/10], Loss: 0.8452
Epoch [8/10], Loss: 0.8006
Epoch [9/10], Loss: 0.7794
Epoch [10/10], Loss: 0.7545


# Evaluate the Model

In [53]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 74.88%


In [55]:
torch.save(model.state_dict(), 'models/cifar10_cnn.pth')